<a href="https://colab.research.google.com/github/Tam1979/TATA-ML/blob/master/w3_1b_session.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Lab 3.1 : Understanding Tensorflow Sessions

### Overview
We will be running Tensorflow in a single node mode.

### Depends On
None

### Run time
20 mins

## STEP 0: To Instructor
Please go through this lab on 'screen' first.

## STEP 1: Login to your Tensorflow node
Instructor will provide details

## STEP 2: Defining some Constants

Let's try to define some constants in tensorflow.  Try defining the following constants



In [0]:
import tensorflow as tf
import numpy as np

If you see an error in executing the cell, you probably don't have tensorflow installed, or are not using the proper virtual environment.  Please see 

In [0]:
# Constant
constant1 = tf.constant(1.0, tf.float32)  #constant value: 1.0
constant2= tf.constant(2.0, tf.float32)   #constant value: 2.0

## STEP 3: Try to see the value of your constants

Try seeing the value of your constants, by running the next cell.

In [0]:
constant1

<tf.Tensor 'Const:0' shape=() dtype=float32>

Was that what you expected?  Where is value 1.0? 

You should see the following output

```console
   <tf.Tensor 'Const:0' shape=() dtype=float32>
```

It turns out that tensorflow constants don't show up this way.  They will only show up when we run the session.  The session consists of a graph of actions.

## Step 4: Run the DAG in Session to see the constants

In [0]:
s = tf.Session()  #Initialize a session
s.run([constant1, constant2])  #Pass in a tensor as a vector with 2 const


[1.0, 2.0]

Aha!  There they are.  You should have seen the following.

```console
[1.0, 2.0]
```

Congradulations!  You have just run your very first tensorflow program.  Albeit a trivially simple one.

## Step 5: Use Add() to add constants together

Let's try to do an actual operation.  We will use the tf.add() to add constants together.

Remember. Nothing will happen until we call run() on our session.

In [0]:
summed = tf.add(constant1, constant2)
summed

<tf.Tensor 'Add:0' shape=() dtype=float32>

In [0]:
s.run(summed)

3.0

Yes! We just found out that 1+2=3!  Isn't tensorflow amazing? ;-)

## Step 6: Use Multiply() to multiply constats together

Ok, now it's your turn.  Use what you learned from add() to do a similar operation on multiply()

In [0]:
# TODO: use tf.multiply() to multiply the constants together.
multiplied = constant1
s.run(constant2)

2.0

## Step 7: See the Default Session Graph

There's a difference between the `Session` and the `InteractiveSession`.  The `InteractiveSession` will make this the default session. 


In [0]:
# Run this "helper" code to visualize the session graph.

from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "b<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

### Run the following cell block:

This will create a new session and show you the results.

In [0]:
sess2 = tf.InteractiveSession()
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a + b
# We can just use 'c.eval()' without passing 'sess'
print(c.eval())

show_graph(tf.get_default_graph())

sess2.close()

11.0


**TODO** Try making your own session, do something, and look at your new session graph:

In [0]:
sess2 = tf.InteractiveSession()

## TODO: Do Something HERE
a = tf.constant(2.0)
b = tf.constant(2.0)
c = a + b
# We can just use 'c.eval()' without passing 'sess'
print(c.eval())

show_graph(tf.get_default_graph())

sess2.close()

show_graph(tf.get_default_graph())

sess2.close()

4.0



## Step 7: Set up a Parameterized Lambda Function

We can set up a parameterized lambda function by using placeholders.  Placeholders are like variables to our funciton.


In [0]:
param1 = tf.placeholder(tf.float32)
param2 = tf.placeholder(tf.float32)

plus = param1 + param2 
s.run(plus, {param1: 1.0, param2: 2.0})

3.0

## Step 8: Define a variable

So far, we have only defined constants.  Constants are values that can never change (immutable). We can set variables to mutable values. These are used in Tensorflow to specify variables.

These are used to maintain state in your application.

In [0]:
sess2.close()
tf.reset_default_graph()

sess2 = tf.InteractiveSession()
my_count = tf.get_variable("my_count", dtype=tf.int32, trainable=False, initializer=tf.constant(0))
sess2.run(tf.initializers.global_variables())
# We can just use 'c.eval()' without passing 'sess'
print(my_count.eval())
increment_my_count = tf.assign(my_count, my_count+1)

print(increment_my_count.eval())

show_graph(tf.get_default_graph())

sess2.close()



Instructions for updating:
Colocations handled automatically by placer.
0
1


## Step 9: Try out Eager Execution

So far, we've explained about Tensorflow's session architecture. This mode, while confusing at first, allows TensorFlow to optimize for parallelization.

However, sometimes we would rather just have the more familiar "instant" evaluation, as we are used to in vanilla python.  This is great for interactive exploratory analysis, in which raw speed is not our primary concern. 

Eager Evaluation also makes it easy to go back and forth between numpy and tensorflow tensors.

**Important** : You should go to the Kernel -> Restart above, or the equivalent "Reset Runtime" on Colaboratory, and then run **only** these cells.  

In [0]:
import tensorflow as tf

tf.enable_eager_execution()

In [0]:
print(tf.add(1, 2))  # Note that the value is immediately evaluated!!
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.encode_base64("hello world"))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))

Tensor("Add_2:0", shape=(), dtype=int32)
Tensor("Add_3:0", shape=(2,), dtype=int32)
Tensor("Square:0", shape=(), dtype=int32)
Tensor("Sum:0", shape=(), dtype=int32)
Tensor("EncodeBase64:0", shape=(), dtype=string)
Tensor("add_4:0", shape=(), dtype=int32)


In [0]:
### Converting between Numpy Arrays and TF Tensors in eager evaluation
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically
Tensor("Mul:0", shape=(3, 3), dtype=float64)
And NumPy operations convert Tensors to numpy arrays automatically
Tensor("add_1:0", shape=(3, 3), dtype=float64)
The .numpy() method explicitly converts a Tensor to a numpy array


AttributeError: ignored